<a href="https://colab.research.google.com/github/KonstantinBurkin/Machine_Learning_Project/blob/main/Machine_Learning_Delivery_Club.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sales Forecasting

In this project I build a Machine Learning method that predicts the number of orders for each store in Delivery Club and each product for the next week. Here, I use data set describing the number of sales of goods in stores on the Delivery Club platform.

Two data sets (train.csv and test.csv) are used to train Regression model and evaluate its accuracy. Both data sets have the same list of variables:
- id - уникальный идентификатор, представляющий связку (product_id,
store_id, date). Это значит, что для каждой тройки (product_id, store_id, date) существует лишь один id, он не повторяется в данных
- date - дата продажи продукта
- city_name - название города, в котором происходила продажа
- store_id - уникальный идентификатор для каждого магазина
- category_id - категория продаваемого товара
- product_id - уникальный идентификатор товара
- price - цена товара
- weather_desc - краткое описание погоды в этом городе в день продажи
- humidity - влажность в этом городе в день продажи
- temperature - температура в этом городе в день продажи
- pressure - атмосферное давление в этом городе в день продажи
- sales - количество продаж товара (это то, что нужно прогнозировать)

In [1]:
# Import libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn


In [6]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

## Data sets description

In [16]:
train.shape

(212768, 12)

In [8]:
train.head()

,id,date,city_name,store_id,category_id,product_id,price,weather_desc,humidity,temperature,pressure
0,666677,2022-02-14,Москва,1,1,1,4.79,облачно,87.3125,-1.9375,749.3125
1,666678,2022-02-15,Москва,1,1,1,4.79,переменная облачность,88.7500,-1.2500,752.6875
2,666679,2022-02-16,Москва,1,1,1,4.79,переменная облачность,90.3750,-1.5625,746.3125
3,666680,2022-02-17,Москва,1,1,1,4.79,"облачно, небольшой дождь",98.0000,1.7500,732.6875
4,666681,2022-02-18,Москва,1,1,1,4.79,"облачно, небольшие осадки",95.5000,1.3750,733.0000


In [11]:
train.describe()

,id,store_id,category_id,product_id,price,humidity,temperature,pressure,sales
count,212768.000000,212767.000000,212767.000000,212767.000000,212767.000000,212767.000000,212767.000000,212767.000000,212767.000000
mean,106384.500000,24.554701,2.436459,17.981412,5.049983,72.964370,6.038753,751.430478,10.527352
std,61420.975375,14.086518,1.931186,10.781425,3.375694,20.410612,11.490579,9.397442,15.020995
min,1.000000,1.000000,1.000000,1.000000,1.930000,13.750000,-24.000000,717.250000,0.000000
25%,53192.750000,13.000000,1.000000,9.000000,3.000000,57.937500,-2.000000,745.312500,2.000000
50%,106384.500000,24.000000,1.000000,17.000000,4.090000,76.812500,5.687500,751.062500,6.000000
75%,159576.250000,37.000000,4.000000,28.000000,6.020000,91.562500,13.187500,758.000000,12.000000
max,212768.000000,48.000000,9.000000,35.000000,18.630000,100.000000,34.250000,779.000000,269.000000


In [17]:
test.shape

(24836, 11)

In [9]:
test.head()

,id,date,city_name,store_id,category_id,product_id,price,weather_desc,humidity,temperature,pressure
0,666677,2022-02-14,Москва,1,1,1,4.79,облачно,87.3125,-1.9375,749.3125
1,666678,2022-02-15,Москва,1,1,1,4.79,переменная облачность,88.7500,-1.2500,752.6875
2,666679,2022-02-16,Москва,1,1,1,4.79,переменная облачность,90.3750,-1.5625,746.3125
3,666680,2022-02-17,Москва,1,1,1,4.79,"облачно, небольшой дождь",98.0000,1.7500,732.6875
4,666681,2022-02-18,Москва,1,1,1,4.79,"облачно, небольшие осадки",95.5000,1.3750,733.0000


In [12]:
test.describe()

,id,store_id,category_id,product_id,price,humidity,temperature,pressure
count,24836.000000,24836.000000,24836.000000,24836.000000,24836.000000,24836.000000,24836.000000,24836.000000
mean,679094.500000,78.053551,2.375423,17.832300,5.201144,87.285168,-0.751719,747.908286
std,7169.679979,45.689019,1.876578,10.826993,3.491933,9.839292,4.059063,9.743387
min,666677.000000,1.000000,1.000000,1.000000,1.930000,55.875000,-10.500000,730.312500
25%,672885.750000,40.000000,1.000000,9.000000,3.000000,84.812500,-3.625000,740.000000
50%,679094.500000,76.000000,1.000000,17.000000,4.090000,89.937500,-0.312500,748.937500
75%,685303.250000,117.000000,4.000000,28.000000,6.020000,94.312500,1.750000,754.687500
max,691512.000000,164.000000,9.000000,35.000000,18.630000,98.625000,9.062500,769.000000


## Building regression model